In [32]:
#https://www.youtube.com/watch?v=APmF6qE3Vjc (17:50)
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

import pandas as pd
import seaborn as sns
from skimage.io import imread, imshow
import cv2
import os

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")
import scipy.misc

from subprocess import check_output
import glob

import PIL
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [33]:
#Setting each path to the folders for each type of cervix
basepath = "/Users/keerat/dev/AOSResearch/resources/Train/"
path_type1 = "/Users/keerat/dev/AOSResearch/resources/Train/Type_1"
path_type2 = "/Users/keerat/dev/AOSResearch/resources/Train/Type_2"
path_type3 = "/Users/keerat/dev/AOSResearch/resources/Train/Type_3"
paths = [path_type1, path_type2, path_type3]

#glob will sort through each folder(Type 1, 2, 3) and return the information in it as a list
type1_cervix_images = glob.glob("/Users/keerat/dev/AOSResearch/resources/Train/Type_1/*.jpg")
type2_cervix_images = glob.glob("/Users/keerat/dev/AOSResearch/resources/Train/Type_2/*.jpg")
type3_cervix_images = glob.glob("/Users/keerat/dev/AOSResearch/resources/Train/Type_3/*.jpg")
test_cervix_images = glob.glob("/Users/keerat/dev/AOSResearch/resources/Test/*.jpg")

#all_cervix_images holds all the data
all_cervix_images = type1_cervix_images + type2_cervix_images + type3_cervix_images + test_cervix_images

#checking the number of each cervix type 
print("The number of Type 1 cervices is {}.".format((len(type1_cervix_images))))
print("The number of Type 2 cervices is {}.".format((len(type2_cervix_images))))
print("The number of Type 3 cervices is {}.".format((len(type3_cervix_images))))
print("The number of Test cervices is {}.".format((len(test_cervix_images))))
print("The total number of all cervices is {}.".format((len(all_cervix_images))))

#The data has been transported inton a dataframe
all_cervix_images = pd.DataFrame({'imagepath': all_cervix_images})
all_cervix_images['filetype'] = all_cervix_images.apply(lambda row: row.imagepath.split(".")[-1], axis=1)
all_cervix_images['type'] = all_cervix_images.apply(lambda row: row.imagepath.split("/")[-2], axis=1)
all_cervix_images.head()

The number of Type 1 cervices is 248.
The number of Type 2 cervices is 780.
The number of Type 3 cervices is 450.
The number of Test cervices is 504.
The total number of all cervices is 1982.


,imagepath,filetype,type
0,/Users/keerat/dev/AOSResearch/resources/Train/...,jpg,Type_1
1,/Users/keerat/dev/AOSResearch/resources/Train/...,jpg,Type_1
2,/Users/keerat/dev/AOSResearch/resources/Train/...,jpg,Type_1
3,/Users/keerat/dev/AOSResearch/resources/Train/...,jpg,Type_1
4,/Users/keerat/dev/AOSResearch/resources/Train/...,jpg,Type_1


In [34]:
sizes = []
sizes_files = []
for filepath in all_cervix_images['imagepath']:
    if(Image.open(filepath).size not in sizes):
        sizes.append(Image.open(filepath).size)
        sizes_files.append(filepath)
for a in sizes_files:
    print(type(a))
    print(Image.open(a).size)

<class 'str'>
(3264, 4160)
<class 'str'>
(3264, 3917)
<class 'str'>
(3120, 4160)
<class 'str'>
(3264, 2448)
<class 'str'>
(2340, 4160)
<class 'str'>
(3096, 4128)
<class 'str'>
(2448, 3264)
<class 'str'>
(480, 640)
<class 'str'>
(2322, 4128)


In [35]:
#3264 = max width
#4160 = max height
changed_size = []
print("Original Sizes (rows, columns)")
for t in sizes_files:
    print(Image.open(t).size)
    if (Image.open(t).size[1] % 2 == 1):
        top = (int)((4160 - Image.open(t).size[1])/2 + 1)
        bottom = (int)((4160 - Image.open(t).size[1])/2)
        left = (int)((3264 - Image.open(t).size[0])/2)
        right = (int)((3264 - Image.open(t).size[0])/2)
    elif (Image.open(t).size[0] % 2 == 1):
        top = (int)((4160 - Image.open(t).size[1])/2)
        bottom = (int)((4160 - Image.open(t).size[1])/2)
        left = (int)((3264 - Image.open(t).size[0])/2 + 1)
        right = (int)((3264 - Image.open(t).size[0])/2)
    else:
        top = (int)((4160 - Image.open(t).size[1])/2)
        bottom = (int)((4160 - Image.open(t).size[1])/2)
        left = (int)((3264 - Image.open(t).size[0])/2)
        right = (int)((3264 - Image.open(t).size[0])/2)
    #changed_size.append(cv2.copyMakeBorder( cv2.imread(t), top, bottom, left, right, cv2.BORDER_CONSTANT))
    changed_size.append(cv2.copyMakeBorder( cv2.imread(t), top, bottom, left, right, cv2.BORDER_CONSTANT, value = [0,0,0]))
    image = Image.fromarray(cv2.copyMakeBorder( cv2.imread(t), top, bottom, left, right, cv2.BORDER_CONSTANT, value = [0,0,0]))
    image.save(t)
print("Zero-padded sizes (columns, rows)")    
for a in changed_size:
    print(np.shape(a))

Original Sizes (rows, columns)
(3264, 4160)
(3264, 3917)
(3120, 4160)
(3264, 2448)
(2340, 4160)
(3096, 4128)
(2448, 3264)
(480, 640)
(2322, 4128)
Zero-padded sizes (columns, rows)
(4160, 3264, 3)
(4160, 3264, 3)
(4160, 3264, 3)
(4160, 3264, 3)
(4160, 3264, 3)
(4160, 3264, 3)
(4160, 3264, 3)
(4160, 3264, 3)
(4160, 3264, 3)


In [36]:
tf.__version__

'1.0.0'

In [37]:
# Convolutional Layer 1.
#When dealing with high-dimensional inputs such as images, 
#it is impractical to connect neurons to all neurons in the previous volume. 
#Instead, we will connect each neuron to only a local region of the input volume. 
#The spatial extent of this connectivity is a hyperparameter called the receptive field 
#of the neuron (equivalently this is the filter size). 
#smaller size than input
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

#more filters, featuer map will b
# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

In [27]:
# We know that MNIST images are 28 pixels in each dimension.
img_size_x = 3264
img_size_y = 4160

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size_x * img_size_y

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size_x, img_size_y)

# Number of colour channels for the images: 1 channel for gray-scale.
#https://en.wikipedia.org/wiki/Channel_(digital_image)
#channels mean number of primary colors
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 3

In [28]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
# Get the first images from the test-set.
data = []
for a in all_cervix_images['imagepath']:
    data.append(plt.imread(a))

# Get the true classes for those images.
cls_true = data.a

# Plot the images and labels using our helper-function above.
plot_images(images=images, cls_true=cls_true)

In [ ]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    #equivalent to y intercept
    #constant value carried over across matrix math
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [ ]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

In [ ]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

In [ ]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [ ]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)